# Combining and Merging Streams

In [1]:
import os
import sys
sys.path.append("../")

In [2]:
from IoTPy.core.stream import Stream, run
from IoTPy.agent_types.op import map_element
from IoTPy.helper_functions.recent_values import recent_values

## Combining Streams with Binary Operators
For streams x, y, and a binary operator, op:
x op y
is a stream whose n-th value is x[n] op y[n].

The following example illustrates how you can combine streams using binary operators such as + and *.

The example after the next one illustrates a functional form for stream definitions.

In [3]:
w = Stream('w')
x = Stream('x')
y = Stream('y')

z = (x+y)*w
# z[n] = (x[n] + y[n])*w[n]
w.extend([1, 10, 100])
x.extend(list(range(10, 20, 1)))
y.extend(list(range(5)))

run()

print ('recent_values of z are:')
print(recent_values(z))


recent_values of z are:
[10, 120, 1400]


In [4]:
from IoTPy.agent_types.basics import fmap_e

# Decorate terminating function to specify non-terminating agent.
@fmap_e
def f(v): return v+10

@fmap_e
def g(w): return w * 2

w = Stream('w')
x = Stream('x')
y = Stream('y')

z = f(x+y)*g(w)
# z[n] = f(x[n]+y[n])*g(w[n])

w.extend([1, 10, 100])
x.extend(list(range(10, 20, 1)))
y.extend(list(range(5)))

run()

print ('recent_values of z are:')
print(recent_values(z))

recent_values of z are:
[40, 440, 4800]


## Examples of zip_stream and zip_map
zip_stream is similar to zip except that zip operates on lists and zip_stream operates on streams.

zip_map applies a specified function to the lists obtained by zipping streams.

In [5]:
from IoTPy.agent_types.merge import zip_stream

def example_of_zip_stream():
    x = Stream('x')
    y = Stream('y')
    z = Stream('z')
    zip_stream(in_streams=[x,y], out_stream=z)

    x.extend(['A', 'B', 'C'])
    y.extend(list(range(100, 1000, 100)))
    run()

    print ('recent values of x are')
    print (recent_values(x))
    print ('recent values of y are')
    print (recent_values(y))
    print ('recent values of z are')
    print (recent_values(z))

example_of_zip_stream()

recent values of x are
['A', 'B', 'C']
recent values of y are
[100, 200, 300, 400, 500, 600, 700, 800, 900]
recent values of z are
[('A', 100), ('B', 200), ('C', 300)]


In [6]:
from IoTPy.agent_types.basics import zip_map

def example_of_zip_map():
    x = Stream('x')
    y = Stream('y')
    z = Stream('z')
    zip_map(func=sum, in_streams=[x,y], out_stream=z)

    x.extend(list(range(5)))
    y.extend(list(range(100, 1000, 100)))
    run()

    print ('recent values of x are')
    print (recent_values(x))
    print ('recent values of y are')
    print (recent_values(y))
    print ('recent values of z are')
    print (recent_values(z))

example_of_zip_map()
    

recent values of x are
[0, 1, 2, 3, 4]
recent values of y are
[100, 200, 300, 400, 500, 600, 700, 800, 900]
recent values of z are
[100, 201, 302, 403, 504]


## Defining Aggregating Functions on Streams
The example below shows how you can create aggregators, such as sum_streams, on streams.

In [7]:
import numpy as np

def merge_function(func, streams):
    out_stream = Stream()
    zip_map(func, streams, out_stream)
    return out_stream

def sum_streams(streams): return merge_function(sum, streams)
def median_streams(streams): return merge_function(np.median, streams)
                                            
w = Stream('w')
x = Stream('x')
y = Stream('y')

sums = sum_streams([w,x,y])
medians = median_streams([w,x,y])
                                            
w.extend([4, 8, 12, 16])
x.extend([0, 16, -16])
y.extend([2, 9, 28, 81, 243])
run()

print ('recent values of sum of streams are')
print (recent_values(sums))
print ('recent values of medians of streams are')
print (recent_values(medians))

recent values of sum of streams are
[6, 33, 24]
recent values of medians of streams are
[2.0, 9.0, 12.0]


## Merging Windows
In the example, 
<br>
merge_window(func=f, in_streams=[x,y], out_stream=z, window_size=2, step_size=2)
<br>
creates windows of window_size and step_size for each of the input streams. Thus the windows for the two input streams are:
<br>
[x[0], x[1]],  [x[2], x[3]],   [x[4], x[5]], ....
<br>
[y[0], y[1]],  [y[2], y[3]],   [y[4], y[5]], ....
<br>
Calls to function f return:
<br>
max([x[0], x[1]]) - min([y[0], y[1]]),   max([x[2], x[3]]) - min([y[2], y[3]]), ...


In [8]:
from IoTPy.agent_types.merge import merge_window
def f(two_windows):
    first_window, second_window = two_windows
    return max(first_window) - min(second_window)

x = Stream('x')
y = Stream('y')
z = Stream('z')

merge_window(func=f, in_streams=[x,y], out_stream=z, window_size=2, step_size=2)

x.extend(list(range(4, 10, 1)))
y.extend(list(range(0, 40, 4)))

run()

print ('recent values of z are')
print (recent_values(z))

recent values of z are
[5, -1, -7]


## Asynchronous Merges
merge_asynch(f, in_streams, out_stream)
<br>
Function f operates on a 2-tuple: an index and a value of an input stream, and f returns a single value which is an element of the output stream.
<br>
Elements from the input streams arrive asynchronously and nondeterministically at this merge agent. The index identifies the input stream on which the element arrived.
<br>
<br>
In this example, the agent merges streams of Fahrenheit and Celsius temperatures to produce an output stream of Kelvin temperatures. The list of input streams is [Fahrenheit, Celsius], and so the indices associated with Fahrenheit and Celsius are 0 and 1 respectively.
<br>
To convert Celsius to Kelvin add 273 and to convert Fahrenheit convert to Celsius and then add 273.

In [9]:
from IoTPy.agent_types.merge import merge_asynch

Fahrenheit = Stream('Fahrenheit')
Celsius = Stream('Celsius')
Kelvin = Stream('Kelvin')

def convert_to_Kelvin(index_and_temperature):
    index, temperature = index_and_temperature
    result = 273 + (temperature if index == 1
                       else (temperature - 32.0)/1.8)
    return result
    
merge_asynch(func=convert_to_Kelvin, 
             in_streams=[Fahrenheit, Celsius], out_stream=Kelvin)

Fahrenheit.extend([32, 50])
Celsius.extend([0.0, 10.0])

run()

Fahrenheit.extend([14.0])
Celsius.extend([-273.0, 100.0])

run()

print ('Temperatures in Kelvin are')
print (recent_values(Kelvin))

Temperatures in Kelvin are
[273.0, 283.0, 273.0, 283.0, 263.0, 0.0, 373.0]


### blend
blend(func, in_streams, out_stream)
<br>
blend executes func on each element of an in_stream when the element arrives at the agent and puts the result on the out_stream. 
<br>
blend is nondeterministic because different executions of a program may results in elements of input streams arriving at the agent in different orders.
<br>
blend is similar to merge_asynch except that in blend func operates on an element of any in_stream whereas in merge_asynch func operates on a pair (index, element) where index identifies the input stream.
<br>
<br>
In this example, func doubles its argument. Initially, the only elements to arrive at the agent are [0, 1 2] on stream x, and so the agent puts [0, 2, 4] on the output stream. Then the next elements to arrive at the agent are [3, 4] also on stream x, and so the agent appends [6, 8] to the output. Then the next elements to arrive at the agent are [100, 110, 120] on stream y, and do the agent extends the output with [200, 220, 240].

In [10]:
from IoTPy.agent_types.merge import blend

def test_blend():
    x = Stream('x')
    y = Stream('y')
    z = Stream('z')

    blend(func=lambda v: 2*v, in_streams=[x,y], out_stream=z)

    x.extend(list(range(3)))
    run()
    print (recent_values(z))

    x.extend(list(range(3, 5, 1)))
    run()
    print (recent_values(z))

    y.extend(list(range(100, 130, 10)))
    run()
    print (recent_values(z))

    x.extend(list(range(5, 10, 1)))
    run()
    print (recent_values(z))

test_blend()
    


[0, 2, 4]
[0, 2, 4, 6, 8]
[0, 2, 4, 6, 8, 200, 220, 240]
[0, 2, 4, 6, 8, 200, 220, 240, 10, 12, 14, 16, 18]


In [11]:
from IoTPy.core.stream import StreamArray
from IoTPy.agent_types.merge import merge_list

def test_merge_list_with_stream_array():
    x = StreamArray()
    y = StreamArray()
    z = StreamArray(dtype='bool')

    # Function that is encapsulated
    def f(two_arrays):
        x_array, y_array = two_arrays
        return x_array > y_array

    # Create agent
    merge_list(f, [x,y], z)

    x.extend(np.array([3.0, 5.0, 7.0, 11.0, 30.0]))
    y.extend(np.array([4.0, 3.0, 10.0, 20.0, 25.0, 40.0]))
    run()

    print('recent values of z are:')
    print (recent_values(z))
test_merge_list_with_stream_array()

recent values of z are:
[False  True False False  True]


In [12]:
from IoTPy.agent_types.merge import timed_zip

def test_timed_zip():
    x = Stream('x')
    y = Stream('y')
    z = Stream('z')

    # timed_zip_agent(in_streams=[x,y], out_stream=z, name='a')
    timed_zip(in_streams=[x, y], out_stream=z)

    x.extend([(1, "A"), (5, "B"), (9, "C"), (12, "D"), (13, "E")])
    y.extend([(5, "a"), (7, "b"), (9, "c"), (12, "d"), (14, 'e'), (16, 'f')])

    run()

    print ('recent values of z are')
    print (recent_values(z))

test_timed_zip()

recent values of z are
[[1, ['A', None]], [5, ['B', 'a']], [7, [None, 'b']], [9, ['C', 'c']], [12, ['D', 'd']], [13, ['E', None]]]


In [13]:
from IoTPy.agent_types.merge import timed_mix

def test_timed_mix_agents():
    x = Stream('x')
    y = Stream('y')
    z = Stream('z')

    timed_mix([x,y], z)

    x.append((0, 'a'))
    run()
    # time=0, value='a', in_stream index is
    assert recent_values(z) == [(0, (0, 'a'))]

    x.append((1, 'b'))
    run()
    assert recent_values(z) == [(0, (0, 'a')), (1, (0, 'b'))]

    y.append((2, 'A'))
    run()
    assert recent_values(z) == \
      [(0, (0, 'a')), (1, (0, 'b')), (2, (1, 'A'))]

    y.append((5, 'B'))
    run()
    assert recent_values(z) == \
      [(0, (0, 'a')), (1, (0, 'b')), (2, (1, 'A')), (5, (1, 'B'))]

    x.append((3, 'c'))
    run()
    assert recent_values(z) == \
      [(0, (0, 'a')), (1, (0, 'b')), (2, (1, 'A')), (5, (1, 'B'))]

    x.append((4, 'd'))
    run()
    assert recent_values(z) == \
      [(0, (0, 'a')), (1, (0, 'b')), (2, (1, 'A')), (5, (1, 'B'))]

    x.append((8, 'e'))
    run()
    assert recent_values(z) == \
      [(0, (0, 'a')), (1, (0, 'b')), (2, (1, 'A')), (5, (1, 'B')), (8, (0, 'e'))]
    print (recent_values(z))

test_timed_mix_agents()

[(0, (0, 'a')), (1, (0, 'b')), (2, (1, 'A')), (5, (1, 'B')), (8, (0, 'e'))]
